# Lasso (coordinate descent) - Carlos and Jeremy - R&I 

We will:
* Develop a function to normalize features and implement coordinate descent for LASSO 
* Study the L1 penalty (regularization)


In [1]:
import pandas as pd
import numpy as np

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 
              'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float,
              'bedrooms':float, 'zipcode':str, 'long':float, 
              'sqft_lot15':float, 'sqft_living':float, 'floors':str, 
              'condition':int, 'lat':float, 'date':str, 
              'sqft_basement':int, 'yr_built':int, 'id':str,
              'sqft_lot':int, 'view':int}

# Load the house sales dataset

Dataset is from house sales in King County, 
the region where the city of Seattle, WA is located.

In [3]:
sales = pd.read_csv('kc_house_data.csv',dtype=dtype_dict)
sales['floors'] = sales['floors'].astype(float).astype(int)

In [4]:
sales.dtypes

id                object
date              object
price            float64
bedrooms         float64
bathrooms        float64
sqft_living      float64
sqft_lot           int32
floors             int32
waterfront         int32
view               int32
condition          int32
grade              int32
sqft_above         int32
sqft_basement      int32
yr_built           int32
yr_renovated       int32
zipcode           object
lat              float64
long             float64
sqft_living15    float64
sqft_lot15       float64
dtype: object

# Plot House Prices from the Dataset

In [5]:
sales['price'].plot()

# Import functions 

In [6]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list 
    # so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list 
    # into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.as_matrix()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.as_matrix()
    return(feature_matrix, output_array)

The `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [7]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing 
    # the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

# Normalize features

We now **normalize features** by dividing each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [8]:
X = np.array([[20.,24.,40.],[22.,28.,44.]])
print X

[[20. 24. 40.]
 [22. 28. 44.]]


In [9]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[29.73213749 36.87817783 59.46427499]


To normalize, apply element-wise division:

In [10]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.67267279 0.65079137 0.67267279]
 [0.73994007 0.7592566  0.73994007]]


Now, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [11]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix,axis = 0)
    normalized_features = feature_matrix/norms
    return normalized_features,norms

Now testing the function:

In [12]:
features, norms = normalize_features(np.array([[20.,24.,40.],[22.,28.,44.]]))
print "Features: " + str(features)

Features: [[0.67267279 0.65079137 0.67267279]
 [0.73994007 0.7592566  0.73994007]]


In [13]:
print "Norms: " + str(norms) 

Norms: [29.73213749 36.87817783 59.46427499]


# Developing the coordinate descent with normalized features

We would like to minimize the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |wt[1]| + ... + |wt[k]|).
```
We do not want to push the intercept to zero so, `wt[0]` is not included in the L1 penalty term.

The **coordinate descent** will be used at each iteration 

We will fix all weights but weight `i` 

Then find the value of weight `i` that minimizes the objective. 

In other words, we seek:
```
argmin_{wt[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |wt[1]| + ... + |wt[k]|) ]
```
where all weights other than `wt[i]` are held to be constant. 

We will also optimize one `wt[i]` at a time, and move through the computation with the weights multiple times. 

For this process:

We use the **cyclical coordinate descent with normalized features**, 
where we cycle through coordinates 0 to (d-1) in order, 
and assume the features were normalized as discussed above. 

We optimize each coordinate using the following formula:
```
       ┌ (Zo[i] + lambda/2)     if Zo[i] < -lambda/2
wt[i] = ├ 0                      if -lambda/2 <= Zo[i] <= lambda/2
       └ (Zo[i] - lambda/2)     if Zo[i] > lambda/2
```
where
```
Zo[i] = SUM[ [feature_i]*(output - prediction + wt[i]*[feature_i]) ].
```

We do no regularize the weight of the constant feature (intercept) `wt[0]`.

With that said, we updated with:
```
wt[0] = Zo[i]
```

# Studying the L1 penalty

Let us consider a simple model with 2 features:

In [14]:
ML1_features = ['sqft_living', 'bedrooms']
ML1_output = 'price'
(ML1_feature_matrix, output) = get_numpy_data(sales, ML1_features, ML1_output)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Normalizing the features:

In [15]:
ML1_feature_matrix, norms = normalize_features(ML1_feature_matrix)

We assign some random set of initial weights and inspect the values of `Zo[i]`:

In [16]:
weights = np.array([2., 8., 6.])

Use `predict_output()` to make predictions on this data.

In [17]:
prediction = predict_output(ML1_feature_matrix,weights)

We compute the values of `Zo[i]` for each feature in this simple model:
```
Zo[i] = SUM[ [feature_i]*(output - prediction + wt[i]*[feature_i]) ]
```

In [18]:
Zo = list()
print weights
for i in range(0,len(ML1_features)+1):
    print i
    feature_i = ML1_feature_matrix[:,i]
    Zo.append( sum((feature_i)*(output - prediction + weights[i]*feature_i) ))
Zo

[2. 8. 6.]
0
1
2


[79400291.53547528, 87939465.18951182, 80966693.92709385]

## Single Coordinate Descent Step

We now minimize the cost function over a single feature `i` 
with the coordinate descent step. 

To optimize over, the function should accept **feature matrix, 
output, current weights, l1 penalty and index of the feature**. 

And the function should return new weight for feature `i`.

In [19]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute Zo[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = feature_matrix[:,i]
    Zo_i = sum((feature_i)*(output - prediction + weights[i]*feature_i) )

    if i == 0: # intercept -- do not regularize
        new_weight_i = Zo_i 
    elif Zo_i < -l1_penalty/2.:
        new_weight_i = Zo_i + l1_penalty/2
    elif Zo_i > l1_penalty/2.:
        new_weight_i = Zo_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function using the following:

In [20]:
# We should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                                                 [2./math.sqrt(13),3./math.sqrt(10)]]), 
                                                 np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.4255588466910251


## Cyclical coordinate descent 

We now implement a cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat. 

We will measure the change in weight for each coordinate. 

If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
* As we loop over features in order and perform coordinate descent, measure how much each coordinate changes.

* After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. 

* Otherwise, go back to step 1.

Return weights

In [21]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, 
                                      initial_weights, l1_penalty, 
                                      tolerance):
    while True:
        maxstep = -1
        maxi = 0
        weights = np.zeros(len(initial_weights))
        for i in range(len(initial_weights)):
            weights[i] = lasso_coordinate_descent_step(i,feature_matrix,
                                                       output,initial_weights,
                                                       l1_penalty)
            step = abs(weights[i]-initial_weights[i])
            if(step>maxstep):
                maxstep = step
                maxi = i          
        initial_weights[maxi]=weights[maxi]
        #print maxi,maxstep
        if maxstep < tolerance:
            return initial_weights

Using the following parameters, learn the weights on the sales dataset. 

In [22]:
sales.dtypes

id                object
date              object
price            float64
bedrooms         float64
bathrooms        float64
sqft_living      float64
sqft_lot           int32
floors             int32
waterfront         int32
view               int32
condition          int32
grade              int32
sqft_above         int32
sqft_basement      int32
yr_built           int32
yr_renovated       int32
zipcode           object
lat              float64
long             float64
sqft_living15    float64
sqft_lot15       float64
constant           int64
dtype: object

In [23]:
ML1_features = ['sqft_living', 'sqft_living15']
ML1_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1

We normalized the feature matrix:

In [24]:
(ML1_feature_matrix, output) = get_numpy_data(sales, 
                                              ML1_features, ML1_output)
(normalized_ML1_feature_matrix, ML1_norms) = normalize_features(ML1_feature_matrix) # normalize features

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Now we execute the LASSO coordinate descent implementation:

In [25]:
weights = lasso_cyclical_coordinate_descent(normalized_ML1_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
RSS = sum((predict_output(normalized_ML1_feature_matrix,weights)-output)**2)
print 'RSS: ' + str(RSS)
print 'Weights: ' + str(weights)

RSS: 1630492383845787.8
Weights: [21624988.74465544 63157256.49484932        0.        ]


# Study the LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [26]:
train_data = pd.read_csv('kc_house_train_data.csv',dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv',dtype=dtype_dict)
train_data['floors'] = train_data['floors'].astype(float).astype(int)
test_data['floors'] = test_data['floors'].astype(float).astype(int)

Let us consider the following set of features.

In [27]:
more_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

Normalize the feature matrix from the TRAINING data with these features:  

In [28]:
(feature_matrix, output) = get_numpy_data(train_data, 
                                          more_features, 'price')
normalized_training,norms = normalize_features(feature_matrix)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Now we learn the weights with `l1_penalty=1e7`, on the training data. 

Initialize weights to all zeros, and set the `tolerance=1`.  

Call resulting weights `wts_1e7`.

In [29]:
initial_weights = np.zeros(14)
wts_1e7 = lasso_cyclical_coordinate_descent(normalized_training, output,
                                            initial_weights, 1e7, 1)
a7 = wts_1e7

In [30]:
a7

array([24429589.93019406,        0.        ,        0.        ,
       48389184.67771954,        0.        ,        0.        ,
        3317511.59607863,  7329960.45195356,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [31]:
print wts_1e7
for i in range(0,len(wts_1e7)):
    if wts_1e7[i]!=0 and i==0:
        print 0,'constant'
    elif wts_1e7[i]!=0:
        print i,more_features[i-1]

[24429589.93019406        0.                0.         48389184.67771954
        0.                0.          3317511.59607863  7329960.45195356
        0.                0.                0.                0.
        0.                0.        ]
0 constant
3 sqft_living
6 waterfront
7 view


Now we learn the weights with `l1_penalty=1e8`, on the training data. 

Initialize weights to all zeros, and set the `tolerance=1`.  

Call resulting weights `wts_1e8`.

In [32]:
initial_weights = np.zeros(14)
wts_1e8 = lasso_cyclical_coordinate_descent(normalized_training, output,
                                            initial_weights, 1e8, 1)

for i in range(0,len(wts_1e8)):
    if wts_1e8[i]!=0:
        if i == 0:
            print 0,'constant'
        else:
            print i,more_features[i-1]
a8 = wts_1e8

0 constant


Finally, we learn the weights with `l1_penalty=1e4`, on the training data. 

Initialize weights to all zeros, and set the `tolerance=5e5`.  

Call resulting weights `wts_1e4`.

In [33]:
initial_weights = np.zeros(14)
wts_1e4 = lasso_cyclical_coordinate_descent(normalized_training, output,
                                            initial_weights, 1e4, 5e5)
for i in range(0,len(wts_1e4)):
    if wts_1e4[i]!=0:
        if i ==0:
            print 0,'constant'
        else:
            print i,more_features[i-1]
a4 = wts_1e4

1 bedrooms
3 sqft_living
4 sqft_lot
5 floors
6 waterfront
7 view
8 condition
9 grade
12 yr_built
13 yr_renovated


## Rescaling learned weights

We can rescale the learned weights to include the normalization.

This will help us to not worry about normalizing the test data: 

With that said, we will scale our weights to make predictions with **original** features:
```
features, norms = normalize_features(features)
weights_normalized = weights / norms
```
Now, we can use the test data, without normalizing it!

We apply to the weights learned above. (`wt_1e4`, `wt_1e7`, `wt_1e8`).

In [34]:
#features, norms = normalize_features(more_features)
normalized_wt_1e4=a4/norms

In [35]:
normalized_wt_1e7=a7/norms

In [36]:
normalized_wt_1e8=a8/norms

To check your results, if you call `normalized_wt_1e7` the normalized version of `wt_1e7`, then:
```
print normalized_wt_1e7[3]
```
should return 161.31745624837794.

In [37]:
print normalized_wt_1e7[3]

161.31749067082325


## Study the learned models on the test data

We will now evaluate the three models on the test data:

In [38]:
(ML1_feature_matrix, ML1_output) = get_numpy_data(test_data, 
                                                  more_features, 'price')

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


The RSS for all three normalized weights on the (unnormalized) `ML1_feature_matrix`:

In [39]:
RSS_wt_4 = sum((predict_output(ML1_feature_matrix,
                               normalized_wt_1e4)-ML1_output)**2)
RSS_wt_7 = sum((predict_output(ML1_feature_matrix,
                               normalized_wt_1e7)-ML1_output)**2)
RSS_wt_8 = sum((predict_output(ML1_feature_matrix,
                               normalized_wt_1e8)-ML1_output)**2)

In [40]:
print 'RSS for wt_1e4: ' + str(RSS_wt_4)
print 'RSS for wt_1e7: ' + str(RSS_wt_7)
print 'RSS for wt_1e8: ' + str(RSS_wt_8)

RSS for wt_1e4: 225789014748755.2
RSS for wt_1e7: 275962057997703.12
RSS for wt_1e8: 537166151497322.4
